In [21]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from transformers import pipeline
import optuna
from transformers import EarlyStoppingCallback
from sklearn.model_selection import train_test_split

In [2]:
BOOKS_PATH = "processed_books.txt"

In [3]:
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

c:\Users\reidp\miniconda3\envs\torch_gpu\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\reidp\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [22]:
def load_and_split_data(file_path, test_size=0.1):
    with open(file_path, "r") as file:
        text = file.read()
    
    stories = text.split("\n\n")
    train_stories, test_stories = train_test_split(stories, test_size=test_size, random_state=42)

    return train_stories, test_stories

In [24]:
train_stories, test_stories = load_and_split_data(BOOKS_PATH)

In [25]:
train_file_path = "train_dataset.txt"
test_file_path = "test_dataset.txt"

with open(train_file_path, "w") as file:
    file.write("\n\n".join(train_stories))
with open(test_file_path, "w") as file:
    file.write("\n\n".join(test_stories))

In [26]:
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=train_file_path,
    block_size=128
)

test_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=test_file_path,
    block_size=128
)

c:\Users\reidp\miniconda3\envs\torch_gpu\lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [5]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False # If set to False, the labels are the same as the inputs with the padding tokens ignored
)

In [6]:
training_args = TrainingArguments(
    output_dir="./output_nlp",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=500,
    save_steps=10_000,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    greater_is_better=False,
    gradient_accumulation_steps=1,
    fp16=True,
    fp16_backend="auto",
    fp16_full_eval=False,
    learning_rate=5e-5,
    weight_decay=0.01,
    adam_beta1=0.9,
    adam_beta2=0.999,
    adam_epsilon=1e-8,
    max_grad_norm=1.0,
    lr_scheduler_type="linear",
    warmup_steps=0,
    label_smoothing_factor=0.0,
    report_to=["tensorboard"],
    seed=42
)

In [7]:
# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)


In [9]:
def generate_story_text(prompt, model_path="./output"):
    generator = pipeline("text-generation", model=model_path, tokenizer="gpt2")
    story_text = generator(prompt, max_length=150, num_return_sequences=1, temperature=0.8, top_k=50, top_p=0.95)[0]["generated_text"]
    return story_text

In [17]:
def objective(trial, train_dataset, data_collator):

    # Define hyperparameters using trial.suggest_* methods
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-3)
    num_train_epochs = trial.suggest_int("num_train_epochs", 1, 5)
    per_device_train_batch_size = trial.suggest_categorical("per_device_train_batch_size", [4, 8, 16])
    weight_decay = trial.suggest_float("weight_decay", 0, 0.1)
    warmup_steps = trial.suggest_int("warmup_steps", 0, 500)

    # Set up the training arguments using the suggested hyperparameters
    training_args = TrainingArguments(
        output_dir="./output",
        overwrite_output_dir=True,
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=per_device_train_batch_size,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        warmup_steps=warmup_steps,
        evaluation_strategy="steps",
        logging_dir="./logs",
        logging_steps=500,
        eval_steps=500,
        save_steps=500,
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="loss",
        greater_is_better=False,
        report_to=["tensorboard"],
        seed=42,
        disable_tqdm=True,
    )

    # Set up the Trainer instance using the suggested hyperparameters
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
    )

    # Train the model and return the best loss
    trainer.train()
    best_loss = trainer.evaluate()["eval_loss"]
    return best_loss

In [20]:
# Create an Optuna study
study = optuna.create_study(direction="minimize", study_name="gpt2_hyperparameter_tuning")

# Optimize the hyperparameters
study.optimize(lambda trial: objective(trial, train_dataset, data_collator), n_trials=20, timeout=3600)

[I 2023-04-17 16:31:10,923] A new study created in memory with name: gpt2_hyperparameter_tuning
C:\Users\reidp\AppData\Local\Temp\ipykernel_11992\1918027171.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-3)
c:\Users\reidp\miniconda3\envs\torch_gpu\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
[W 2023-04-17 16:32:07,407] Trial 0 failed with parameters: {'learning_rate': 0.00047807619038330284, 'num_train_epochs': 5, 'per_device_train_batch_size': 4, 'weight_decay': 0.00945210956957634, 'warmup_steps': 355} be

{'loss': 5.0196, 'learning_rate': 0.0004775795501224805, 'epoch': 0.02}


ValueError: Trainer: evaluation requires an eval_dataset.

In [ ]:
best_params = study.best